In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter to /tmp/pip-req-build-j2y47s02
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter /tmp/pip-req-build-j2y47s02
  Resolved https://github.com/andreinechaev/nvcc4jupyter to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <sys/time.h>
#include <ctime>

__global__ void Primesentrephi(int phi_n,long long int e, long long int D){
    
    long long int thread_id = blockIdx.x * blockDim.x + threadIdx.x;

    if(thread_id <= phi_n){
          if((thread_id*e) % phi_n == 1){

              if(D == 0){
                D = thread_id;
                printf("D = %lld\n", D);
              }
          }
      }
}

__global__ void primes_in_range(int *result)
{
  int number = 10 + (blockIdx.x * blockDim.x) + threadIdx.x;
    if (number >= 200000)
        return;

    if (number % 2 == 0) return;
    for (int divisor = 3; divisor < (number / 2); divisor += 2) {
        if (number % divisor == 0){
            return;
        }
    }
  printf("%d ",number);
    atomicAdd(result, 1);
}

int main(){

  long long int numero; 
  long long int primo [100000]; //vetor q vai aramazenar todos os numero primos
  int i, j, cont;
  long long int p, q; //variaveis para armazenar dois primos (para gerar a chave publica)
  long long int n; //Multiplicação de p e q
  long long int phi_n;
  long long int e;
  long long int d;
  int tam;
  int flag;

  int *result;
    cudaMallocManaged(&result, sizeof(int));
    *result = 0;

  auto begin = std::clock();

    primes_in_range<<<200, 1024>>>(result);
    cudaDeviceSynchronize();

  auto end = std::clock();
  auto duration = double(end - begin) / CLOCKS_PER_SEC * 1000;
    
    printf("\nQuantidade de numeros primos =  %d\n", *result);
  printf("Tempo de execução para descobrir os numeros primos: %d segundos\n", static_cast<int>(duration)); 


  p = 193847;
  q = 199049;

  //DESCOBRINDO A CHAVE PUBLICA
  printf("P = %lld - Q = %lld \n", p, q); 

  n = p * q;
  printf("N = %lld\n", n);

  phi_n = (p-1) * (q-1);
  printf("Phi(N) = %lld\n", phi_n);

  e = 0;
  //procura um numero primo de Phi(N) -> [numeros primos entre si são aqueles q não possuem um divisor em comum, alem do 1. Por exemplo, 25 e 9 são primos entre si]
  for(i = 10; i < phi_n; i++)
  {
      for(j = 2; j < (i + 1); j++)
      {
          if(phi_n % j == 0 && i % j == 0)
              break;

          if(j == i)
              e = i;
      }

      //E vai receber o primeiro numero primo com Phi(n) [o algoritimo em si é um numero aleatorio mas o primeiro serve]
      if(e != 0)
          break;
  }

  printf("E = %lld\n", e);

  printf("CHAVE PUBLICA: [%lld, %lld]\n", n, e);
  
  long long int D = 0;
  
  begin = std::clock();

  Primesentrephi<<<phi_n/1024,1024>>>(phi_n, e,D);


  end = std::clock();
  duration = double(end - begin) / CLOCKS_PER_SEC * 1000;

  printf("Tempo de execução para calcular a chave privada: %d segundos\n", static_cast<int>(duration));

  printf("CHAVE PRIVADA: [%lld]\n", D);

  cudaFree(result);

    return 0;
}

In [ ]:
!nvcc -o teste teste.cu -run

Streaming output truncated to the last 5000 lines.
TO NO PRIMO PHI 568697
TO NO PRIMO PHI 568698
TO NO PRIMO PHI 568699
TO NO PRIMO PHI 568700
TO NO PRIMO PHI 568701
TO NO PRIMO PHI 568702
TO NO PRIMO PHI 568703
TO NO PRIMO PHI 568448
TO NO PRIMO PHI 568449
TO NO PRIMO PHI 568450
TO NO PRIMO PHI 568451
TO NO PRIMO PHI 568452
TO NO PRIMO PHI 568453
TO NO PRIMO PHI 568454
TO NO PRIMO PHI 568455
TO NO PRIMO PHI 568456
TO NO PRIMO PHI 568457
TO NO PRIMO PHI 568458
TO NO PRIMO PHI 568459
TO NO PRIMO PHI 568460
TO NO PRIMO PHI 568461
TO NO PRIMO PHI 568462
TO NO PRIMO PHI 568463
TO NO PRIMO PHI 568464
TO NO PRIMO PHI 568465
TO NO PRIMO PHI 568466
TO NO PRIMO PHI 568467
TO NO PRIMO PHI 568468
TO NO PRIMO PHI 568469
TO NO PRIMO PHI 568470
TO NO PRIMO PHI 568471
TO NO PRIMO PHI 568472
TO NO PRIMO PHI 568473
TO NO PRIMO PHI 568474
TO NO PRIMO PHI 568475
TO NO PRIMO PHI 568476
TO NO PRIMO PHI 568477
TO NO PRIMO PHI 568478
TO NO PRIMO PHI 568479
TO NO PRIMO PHI 568768
TO NO PRIMO PHI 568769
TO NO 

In [ ]:
!nvprof ./teste